In [1]:
import numpy as np
import cPickle
import nltk
from nltk.corpus import stopwords
from collections import Counter
from sklearn.metrics.pairwise import pairwise_distances
import itertools
import string
from nltk.tokenize import sent_tokenize
from nltk.stem.porter import *
import re
#import pyemd
#loading word embeddings
#wv = KeyedVectors.load_word2vec_format("../w2v/w2vemb.bin", binary=True)

# Visualize functions

In [2]:
def write_file(n_clusters_, labels, corpus, mapping_dict, test_corpus):
    for indice_cluster in range(n_clusters_):
        # print indice_cluster
        idx_list = np.where(labels == indice_cluster)[0]
        f1 = open('cluster_%s/%dth_cluster.txt' % (question_class, indice_cluster), 'w+')
        for idx in idx_list:
            f1.write('%s %s\n' % (corpus[mapping_dict[idx] - 1], test_corpus[idx]))
    f1.close()


def write_ne(nes, question_class, mapping_dict):
    f1 = open('ne_%s.txt' % question_class, 'w+')
    for idx, doc in enumerate(nes):
        f1.write('%s %s\n' % (mapping_dict[idx], ' '.join(doc)))
    f1.close()    

def wmd_compute(x1, x2):
    return wv.wmdistance(x1, x2)

def analyze_cluster(n_clusters_, labels, corpus, mapping_dict, test_corpus, filename):
    num = 0
    other_content = []
    for indice_cluster in range(n_clusters_):
        idx_list = np.where(labels == indice_cluster)[0]
        if len(idx_list) > 4:
            f1 = open('%s_%s/%dth_cluster.txt' % (filename, question_class, indice_cluster), 'w+')
            for idx in idx_list:
                f1.write('%s %s\n' % (corpus[mapping_dict[idx] - 1], test_corpus[idx]))
            f1.close()
        if len(idx_list) < 4:
            num = num + len(idx_list)
            for idx in idx_list:
                other_content.append(corpus[mapping_dict[idx] - 1])
    f1 = open('%s_%s/other_cluster.txt' % (filename, question_class), 'w+')
    for doc in other_content:
        f1.write('%s\n' % doc)
    f1.close()
    print num

# Reading Raw text

In [3]:
question_class = 'q10'
fname = '..//data//%s.txt' % question_class
with open(fname) as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content]
print 'length of comments', len(content)
f.close()

length of comments 445


In [4]:
def clean_corpus(x1):
    x_new = []
    for x in x1:
        if 'no improvement' in x:
            continue
        else:
            x_new.append(x)
    return x_new

def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Every dataset is lower cased except for TREC
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)     
    string = re.sub(r"\'s", " ", string) 
    string = re.sub(r"\'ve", " ", string) 
    string = re.sub(r"n\'t", " n\'t", string) 
    string = re.sub(r"\'re", " ", string) 
    string = re.sub(r"\'d", " \'d", string) 
    string = re.sub(r"\'ll", " \'ll", string) 
    string = re.sub(r",", " ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " ", string) 
    string = re.sub(r"\)", " ", string) 
    string = re.sub(r"\?", " ", string) 
    string = re.sub(r"\s{2,}", " ", string)    
    return string.strip().lower()


def process_corpus(x1,stopwords):
    # split three categories: 1 no improvemnt 2 with noun 3 others
    doc_noimprove = []
    clean_sen = []
    org_sen = []
    for x in x1:
        if 'no improvement' in x or 'respondent' in x:
            doc_noimprove.append(x)
        else:
            sents = sent_tokenize(x)
            sen_list = []
            for sent in sents:
                word_list = nltk.word_tokenize(sent)
                word_list = [clean_str(word) for word in word_list if word not in stop_words]
                sen_list = sen_list + word_list
            sen_list = ' '.join(sen_list)
            
            clean_sen.append(sen_list.strip())
            org_sen.append(x)
    return clean_sen, org_sen

def should_nn_v(x1, pos_nn, pos_vv):
    pos_new = nltk.pos_tag(nltk.word_tokenize(x1))
    nn_list = []
    vv_list = []
    for token in pos_new:
        if token[1] in pos_nn: #and token[0] in wv.vocab:
            nn_list.append(token[0])
        if token[1] in pos_vv:
            vv_list.append(token[0])
    print nn_list
    print vv_list
    print x1
    
nn_corpus = []

stop_words = stopwords.words('english')
rule_10_words = ['follow', 'up','call', 'calls', 'give']
stop_words = stop_words + rule_10_words
new_content, org_content = process_corpus(content, stop_words)

#pos_nn = ['NN', 'NNS']
#pos_vv = ['VB']
#print 'no comments', len(doc1)
#print 'single sentence without nn', len(doc3)
#print 'multi-sentence', len(doc4)
#for doc_sing in doc2:
    #should_nn_v(doc_sing, pos_nn, pos_vv)
    

# Representation BOW

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(min_df=2, stop_words='english')

tfidf = tfidf_vectorizer.fit_transform(new_content)



# Representation NMF

In [6]:
from sklearn.decomposition import NMF
n_topics = 50
nmf_data = NMF(n_components=n_topics, random_state=1, alpha=.1, l1_ratio=.5).fit_transform(tfidf)
print nmf_data.shape

(426L, 50L)


# Representation Topic Model

In [7]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

from sklearn.decomposition import LatentDirichletAllocation
n_topics = 20
tm_model = LatentDirichletAllocation(n_topics, max_iter=100,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0).fit(tfidf)
print("\nTopics in LDA model:")
n_top_words=10
tf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(tm_model, tf_feature_names, n_top_words)
tm_data = tm_model.transform(tfidf)


Topics in LDA model:
Topic #0:
touch customers complaint 60 fallow km making available provide responding
Topic #1:
improved vehicle service condition know servicing car check completed regarding
Topic #2:
issue mentioned resolve executive knowledge charges issues customer servicing service
Topic #3:
different sent self asking questions work vehicle called feedback received
Topic #4:
parts proper irritating concentrate fallow periodic taken completing actually number
Topic #5:
time feedback information servicing customers ask busy calling situation long
Topic #6:
parts properly attention proper implement complete 24 customer work check
Topic #7:
servicing car change visit correctly tell mail relationship information road
Topic #8:
option inform felt better explain tell improve need received car
Topic #9:
responding calling dealership mail need process making suppose improve tell
Topic #10:
email question send post mail problem complaint told receive customer
Topic #11:
months busy bet

# Clustering

In [8]:
from sklearn.cluster import KMeans
def print_cluster_docs(model, content):
    file = open('result.txt', 'w')
    for topic_idx in range(model.n_clusters):
        print("Topic #%d:" % topic_idx)
        file.write("Topic #%d:\n" % topic_idx)
        print '\n'
        for idx, label in enumerate(model.labels_):
            if label == topic_idx:
                print content[idx]
                print '\n'
                file.write("%s\n" % content[idx])
    file.close()
true_k = 10
km = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
km.fit(nmf_data)
print_cluster_docs(km, org_content)

Topic #0:


after completing car service , the dealership staff should call the customer next day to back check whether the service was done correctly or not also . avoid checking the after one week or after 10 days .


after 15 days of servicing of the car they should give follow up calls .


they should call and ask if there is any problem or the car is running properly or not . they should do this call within 15 20 days .


they should give the follow up calls frequently like after every one month or forty five days .


they said us that they will contact me to their seniors but their senior 's phone is witched off for next 3 4 days . we contact with their seniors after 2 4 days but that time customer is not in mood . the work is pending only which is suppose to do .


after servicing the car they should give follow up calls within 2 days so that if there is any problem we can share it with them .


they should call to the customers within one or two days . it means after servicing 



they should resolve the issues which customer mentioned on follow up calls because , they are just listening the issue and not improving it .


we don \? t get follow up from nanavati . they should do step by step follow up to customer from time . we handover vehicle to them for servicing on what stage our car is how much work is completed in service of our vehicle . when insurance will be completed this all information should provide by them . we not need to ask them .


they should do the follow up calls should know the condition of our car .


i get the follow up calls but when we show our issue or concern to customer care they don \? t follow up sort out that issue . they follow up same like it sector . we just tell them our issue but they don \? t do anything they just ask us issue . if we suggest something they do n't do implement on it .


wasan toyota always calls me but from shinrai toyota i have hardly received 2 3 calls till now


after completing the periodic service they

In [9]:
print km.labels_

[6 6 6 6 6 4 1 5 3 4 0 6 9 6 6 3 5 6 6 6 6 6 6 9 6 0 3 5 5 6 6 9 3 5 6 6 6
 6 3 6 0 3 5 4 1 1 6 6 6 6 5 9 5 6 3 6 1 7 6 0 6 1 6 3 8 0 1 6 6 4 1 6 3 6
 6 5 6 1 4 6 9 6 1 5 6 6 1 6 1 1 9 6 1 6 4 6 6 1 6 6 1 1 6 4 0 6 6 4 4 1 1
 5 5 4 6 7 5 4 4 6 6 6 5 4 6 6 6 6 6 6 6 6 1 6 1 6 5 6 5 6 6 4 5 6 6 6 6 6
 6 6 0 6 6 6 4 1 6 6 0 6 3 6 4 1 6 5 6 4 6 6 6 6 4 5 4 1 6 6 3 6 5 6 6 6 5
 1 0 1 4 6 6 1 5 0 4 6 6 6 9 5 6 6 5 0 6 3 4 5 0 1 6 9 6 6 1 1 6 5 6 1 6 6
 5 5 1 7 5 5 6 5 4 6 6 5 1 6 5 5 4 6 5 5 4 6 5 6 5 6 6 1 0 5 4 1 7 6 5 6 6
 6 4 0 4 1 1 1 5 4 6 4 6 6 6 6 5 6 6 4 6 6 6 2 1 5 7 5 9 2 6 3 6 6 6 6 5 5
 1 6 5 0 1 5 6 6 6 5 5 6 6 4 6 5 1 1 6 1 5 6 6 9 3 0 6 6 6 1 6 5 6 0 6 4 4
 6 1 6 6 6 6 4 6 6 6 6 6 5 6 6 6 6 8 6 1 6 6 6 5 3 5 5 6 5 8 2 1 9 2 6 6 6
 1 4 8 5 6 5 4 5 6 1 2 5 2 6 1 1 5 4 6 9 6 6 9 5 8 1 1 1 9 6 1 6 6 1 5 6 1
 6 6 1 6 6 1 4 1 5 6 5 2 5 2 1 5 5 1 6]
